# Sparkify Project Workspace
This workspace contains a tiny subset (128MB) of the full dataset available (12GB). Feel free to use this workspace to build your project, or to explore a smaller subset with Spark before deploying your cluster on the cloud. Instructions for setting up your Spark cluster is included in the last lesson of the Extracurricular Spark Course content.

You can follow the steps below to guide your data analysis and model building portion of this project.

In [1]:
# import libraries
import os

from pyspark.sql import SparkSession;
from pyspark.sql.functions import regexp_replace, avg, isnan, countDistinct, count,when,col, concat, desc, explode, lit, min, max, split, udf, isnull;
from pyspark.sql.types import IntegerType;

from pyspark.ml.feature import RegexTokenizer, CountVectorizer,IDF, StringIndexer,VectorAssembler, Normalizer, StandardScaler;
from pyspark.ml import Pipeline;
from pyspark.ml.classification import LogisticRegression, RandomForestClassifier, GBTClassifier;
from pyspark.ml.evaluation import MulticlassClassificationEvaluator;

from pyspark.sql import Window
from pyspark.sql.functions import sum as Fsum

import re;
import datetime;

import matplotlib.pyplot as plt;
import pandas as pd;
import seaborn as sns;
import numpy as np;
from itertools import cycle, islice;

In [2]:
# create a Spark session
spark = SparkSession\
                    .builder\
                    .master('local')\
                    .appName("Sparkify Project")\
                    .getOrCreate();

# Load and Clean Dataset
In this workspace, the mini-dataset file is `mini_sparkify_event_data.json`. Load and clean the dataset, checking for invalid or missing data - for example, records without userids or sessionids. 

In [3]:
#load data "mini_sparkify_event_data.json"
df = spark.read.json('mini_sparkify_event_data.json');

#see top records in dataset
df.persist()

DataFrame[artist: string, auth: string, firstName: string, gender: string, itemInSession: bigint, lastName: string, length: double, level: string, location: string, method: string, page: string, registration: bigint, sessionId: bigint, song: string, status: bigint, ts: bigint, userAgent: string, userId: string]

In [4]:
df.head()

Row(artist='Martha Tilston', auth='Logged In', firstName='Colin', gender='M', itemInSession=50, lastName='Freeman', length=277.89016, level='paid', location='Bakersfield, CA', method='PUT', page='NextSong', registration=1538173362000, sessionId=29, song='Rockpools', status=200, ts=1538352117000, userAgent='Mozilla/5.0 (Windows NT 6.1; WOW64; rv:31.0) Gecko/20100101 Firefox/31.0', userId='30')

In [5]:
print(df.take(5));

[Row(artist='Martha Tilston', auth='Logged In', firstName='Colin', gender='M', itemInSession=50, lastName='Freeman', length=277.89016, level='paid', location='Bakersfield, CA', method='PUT', page='NextSong', registration=1538173362000, sessionId=29, song='Rockpools', status=200, ts=1538352117000, userAgent='Mozilla/5.0 (Windows NT 6.1; WOW64; rv:31.0) Gecko/20100101 Firefox/31.0', userId='30'), Row(artist='Five Iron Frenzy', auth='Logged In', firstName='Micah', gender='M', itemInSession=79, lastName='Long', length=236.09424, level='free', location='Boston-Cambridge-Newton, MA-NH', method='PUT', page='NextSong', registration=1538331630000, sessionId=8, song='Canada', status=200, ts=1538352180000, userAgent='"Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/37.0.2062.103 Safari/537.36"', userId='9'), Row(artist='Adam Lambert', auth='Logged In', firstName='Colin', gender='M', itemInSession=51, lastName='Freeman', length=282.8273, level='paid', location='Ba

In [6]:
#lets look at shema of data :
df.printSchema();

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: long (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)



In [7]:
# number of rows in the dataframe
df.count()

286500

In [8]:
# number of columns in the dataframe
len(df.columns)

18

In [9]:
print("Number of  Rows in the dataframe:{}".format(df.count()));
print("Number of  columns in the dataframe:{}".format(len(df.columns)));
print("columns Present in data set in the dataframe{}".format(df.columns));

Number of  Rows in the dataframe:286500
Number of  columns in the dataframe:18
columns Present in data set in the dataframe['artist', 'auth', 'firstName', 'gender', 'itemInSession', 'lastName', 'length', 'level', 'location', 'method', 'page', 'registration', 'sessionId', 'song', 'status', 'ts', 'userAgent', 'userId']


In [10]:
#discribing all the columns in dataframe
df.describe().toPandas().transpose()

,0,1,2,3,4
summary,count,mean,stddev,min,max
artist,228108,551.0852017937219,1217.7693079161374,!!!,ÃÂlafur Arnalds
auth,286500,None,None,Cancelled,Logged Out
firstName,278154,None,None,Adelaida,Zyonna
gender,278154,None,None,F,M
itemInSession,286500,114.41421291448516,129.76726201140994,0,1321
lastName,278154,None,None,Adams,Wright
length,228108,249.1171819778458,99.23517921058361,0.78322,3024.66567
level,286500,None,None,free,paid
location,278154,None,None,"Albany, OR","Winston-Salem, NC"


After cleaning the dataset, we now checking for invalid or missing data. To illustrate, records without userids or session id's

In [11]:
# check null values in columns(userId and sessionId has none)
df.toPandas().isnull().sum()

artist           58392
auth                 0
firstName         8346
gender            8346
itemInSession        0
lastName          8346
length           58392
level                0
location          8346
method               0
page                 0
registration      8346
sessionId            0
song             58392
status               0
ts                   0
userAgent         8346
userId               0
dtype: int64

In [ ]:
# sum all null items
df.toPandas().isnull().sum().sum()

In [ ]:
# inspect userId column for the first 20 columns
df.select('userId').dropDuplicates().sort('userId').show(20)


In [ ]:
# drop empty strings for userId
df = df.filter(df['userId'] != '')

# Exploratory Data Analysis
When you're working with the full dataset, perform EDA by loading a small subset of the data and doing basic manipulations within Spark. In this workspace, you are already provided a small subset of data you can explore.

### Define Churn

Once you've done some preliminary analysis, create a column `Churn` to use as the label for your model. I suggest using the `Cancellation Confirmation` events to define your churn, which happen for both paid and free users. As a bonus task, you can also look into the `Downgrade` events.

### Explore Data
Once you've defined churn, perform some exploratory data analysis to observe the behavior for users who stayed vs users who churned. You can start by exploring aggregates on these two groups of users, observing how much of a specific action they experienced per a certain time unit or number of songs played.

#### Churn is an event that occurs when a user discontinues a subscription from a business.

## Preliminary Exploratory Data analysis

##### Using page =='Cancellation Confirmation' & auth =='Cancelled' we can deduce our Target Variable ie. if users churn or not

In [ ]:
df.select('page','UserId').groupby('page').agg({'page':'count'}).select('page','count(page)').show()

In [ ]:
df.select('auth','UserId').groupby('auth').agg({'auth':'count'}).select('auth','count(auth)').show()

## Observations:

This dataframe groupings above shows the same value counts for cancelled and cancellation confirmation attribute. This is a confirmation that there is no userId with only one factor. Users with cancellation confirmation requests also executed the cancelled feature.

These are good indicators to easily identify churned users. In summary, we could define churned users as those users who have visited the ‘Cancelation Confirmation’ page and also definitely visited 'Cancelled' page. In addition, we could take the ‘Submit Downgrade’ column into consideration.

### Label to caterorize targets(churn or not)



Next, let us define a churn column by using a user defined function which labels all churned users with a 1 and 0 if otherwise:

In [ ]:
def label(df):
    """
    Function to create a dataframe containing targets column(0-churn and 1-non-churn) and respective counts
    Input 
    df: spark DataFrame
    Output 
    df : contains labelled targets with the churn column
    """
   
    label_df=df.withColumn('churn',when(col('page')=='Cancellation Confirmation', 1).otherwise(0)).\
                            groupBy('userId').agg(Fsum('churn').alias('churn'))
    print(label_df.groupBy('churn').count().show())
    churned_rate = label_df.select(avg("churn")).collect()[0]["avg(churn)"]
    print(f"percentage churn: {churned_rate * 100:,.2f}%")

    df = df.join(label_df, on='userId')
    return df

In [ ]:
df = label(df)

## Analyzing users that stayed versus users that churned using certain features

In [ ]:
#EDA using various columns
def exploratory_data_analysis(df,col_name):
    """
    prints countplot of input col
    Input 
    df: spark DataFrame
    col_name : the column with counted values
    Output :
    None
    
    """
    pd = df.select(["userId", col_name, "churn"]).toPandas()
    sns.countplot(x=col_name, hue="churn", data=pd);

In [ ]:
# exploratory data analysis per gender
exploratory_data_analysis(df,"gender")

##### From the visulaization above more male users churn than female users

In [ ]:
# exploratory data analysis per level
exploratory_data_analysis(df,"level")

##### From the visualization above users that paid for the services have higher churn rate than users that access services for free

In [ ]:
# location name too long;  extract states from its respective location
df = df.withColumn("location", regexp_replace("location", '[^a-zA-Z0-9\,\-]', ''))\
        .withColumn('location', split(col('location'),',').getItem(1))
df.select('location').show(n=10)

In [ ]:
# exploratory data analysis per location
df_state_count= df.select('userId','location','churn').groupby('location','churn').count().toPandas().sort_values(by='count')
fig = plt.figure(figsize=(15,8))
ax = fig.gca()
df_state_count.pivot(index='location', columns='churn', values='count').plot(kind='bar', ax=ax)

##### From the visulization above, some locations have churned users while others have no churned users at all.

In [ ]:
# Lets visualize the gender distribution for number of songs played per session for churned and active users
total_songs_df = df.where('page == "NextSong"').groupby(['churn', 'userId', 'sessionId', 'gender']).count()\
    .groupby(['churn', 'userId', 'gender']).agg({'count': 'avg'})\
    .withColumnRenamed('avg(count)', 'average_played_songs').toPandas()

ax = sns.violinplot('churn', y = 'average_played_songs', hue = 'gender', data = total_songs_df)



###### From the visualization above, we can clearly observe that the churned users played less number of songs per session when compared to active users 

In [ ]:
# Let's visualize the gender distribution for total number of operations per session between normal and churned users
df_ops = df.groupby("churn", "userId", "sessionId", 'gender').count().toPandas()
sns.boxplot(x = 'count', y = 'churn', orient = 'horizontal', data = df_ops, hue = 'gender')

###### From the visualization above, we can clearly observe that churned users performed lesser number of operations compared to active users. Also male users display higher churn rate than female users.

###  "Submit Downgrade" is an indicator for Users Churn or not.¶

In [ ]:
#EDA using downgrade events
df.filter((col('page')=='Downgrade')&(col('churn')==1)).select('userId').distinct().count()

##### Out of the 52 churned users 35 downgraded their service

In [ ]:
df.filter((col('page')=='Submit Downgrade')&(col('churn')==1)).select('userId').distinct().count()

##### Now, observing the submit downgrade feature, only  9 users submitted downgrade out of 52 churned users. This is an indicator that active users submits service downgrade than churned users.

In [ ]:
df.filter((col('page')=='Submit Downgrade')&(col('churn')==0)).select('userId').distinct().count()

# Feature Engineering
Once you've familiarized yourself with the data, build out the features you find promising to train your model on. To work with the full dataset, you can follow the following steps.
- Write a script to extract the necessary features from the smaller subset of data
- Ensure that your script is scalable, using the best practices discussed in Lesson 3
- Try your script on the full data set, debugging your script if necessary

If you are working in the classroom workspace, you can just extract features based on the small subset of data contained here. Be sure to transfer over this work to the larger dataset when you work on your Spark cluster.

# Modeling
Split the full dataset into train, test, and validation sets. Test out several of the machine learning methods you learned. Evaluate the accuracy of the various models, tuning parameters as necessary. Determine your winning model based on test accuracy and report results on the validation set. Since the churned users are a fairly small subset, I suggest using F1 score as the metric to optimize.

# Final Steps
Clean up your code, adding comments and renaming variables to make the code easier to read and maintain. Refer to the Spark Project Overview page and Data Scientist Capstone Project Rubric to make sure you are including all components of the capstone project and meet all expectations. Remember, this includes thorough documentation in a README file in a Github repository, as well as a web app or blog post.